# Python timings

Goal:

* Evaluate the performance of Cython and Numba compared to standard python

First run the test

In [10]:
!pytest test.py -v

============================= test session starts ==============================
platform linux -- Python 3.7.3, pytest-5.0.1, py-1.8.0, pluggy-0.12.0 -- /home/ovewh/anaconda3/bin/python
cachedir: .pytest_cache
rootdir: /home/ovewh/Documents/FYS4150/Computilus/Project2/src/python
plugins: openfiles-0.3.2, arraydiff-0.3, doctestplus-0.3.0, remotedata-0.3.1
collected 9 items                                                              

test.py::test_MaxElemOffDiag[prog0] PASSED                               [ 11%]
test.py::test_MaxElemOffDiag[prog1] PASSED                               [ 22%]
test.py::test_MaxElemOffDiag[prog2] PASSED                               [ 33%]
test.py::test_rotation[prog0] PASSED                                     [ 44%]
test.py::test_rotation[prog1] PASSED                                     [ 55%]
test.py::test_rotation[prog2] PASSED                                     [ 66%]
test.py::test_Eigenvalues[prog0-225] PASSED                              [ 77%]
t

* To evaluate the performance of we will use the profiling tools included in the jupyter suite


In [2]:
import pythonJacobi
import numbaJacobi
import cythonJacobi
import numpy as np
import time
#Defining the size of our test matrices
nDims = [5,10, 20 ,50, 100 ,200]

In [7]:


def run(prog, A, N,timings=None ,runs = 10):
    A_c = A.copy()
    if runs == 10:
        timings = timings
        runs = runs -1
        run(prog,A_c,N, timings, runs=runs)
    elif (runs > 0):
        runs = runs -1
        run(prog,A_c,N, timings, runs=runs)
        
#     iterations = 0
    A_c = A.copy()
    t0 = time.time()
    offDiagMax = prog.maxElemOffDiag(A,N)[0]
    while(offDiagMax > 10e-9):
#         iterations += 1
        offDiagMax, row, col = prog.maxElemOffDiag(A, N)
        A = prog.jacobiRotate(A,row, col, N)

    t1 = time.time()
    t = t1-t0
    timings.append(t)
    
    
    

In [8]:
cythonavgTime = []
testMarices = []

for nDim in nDims:
    h = 1./(nDim)
    d = 2./(h**2)
    a = -1./(h**2)
    testMarices.append(cythonJacobi.create_toeplitz(d, a, nDim))
for i in range(len(nDims)):
    timings = []
    run(cythonJacobi, testMarices[i], nDims[i], timings)
    cythonavgTime.append(np.mean(timings))
    
cythonavgTime

[0.00027456283569335935,
 0.0012024641036987305,
 0.03915026187896729,
 4.309332394599915]

In [9]:
testMarices = []
numbaAvgTime = []

for nDim in nDims:
    h = 1./(nDim)
    d = 2./(h**2)
    a = -1./(h**2)
    testMarices.append(numbaJacobi.create_toeplitz(d, a, nDim))

for i in range(len(nDims)):
    timings = []
    run(numbaJacobi, testMarices[i], nDims[i], timings)
    numbaAvgTime.append(np.mean(timings))
numbaAvgTime

[0.03364124298095703,
 0.00013506412506103516,
 0.01658651828765869,
 3.452531361579895]

In [16]:
testMarices = []
pythonAvgTime = []

for nDim in nDims:
    h = 1./(nDim)
    d = 2./(h**2)
    a = -1./(h**2)
    testMarices.append(pythonJacobi.create_toeplitz(d, a, nDim))

for i in range(len(nDims[0:3])):
    timings = []
    run(pythonJacobi, testMarices[i], nDims[i], timings)
    pythonAvgTime.append(np.mean(timings))    


In [18]:
pythonAvgTime[2]/numbaAvgTime[2]

888.1060833217142